# Обмен данными (Лабораторная работа №4, Вариант 24)
## Выполнил: студент группы БВТ2001 Хайдаров Адель Фанисович
##### Цель работы:
* изучение набора средств коммуникации процессов в Windows NT 
* получение практических навыков по использованию WinAPI для программирования механизмов межпроцессного взаимодействия
получение практических навыков использования Win32 API для синхронизации потоков.
##### Задание: Реализовать алгоритм блочной сортировки файла целых чисел. Каждый процесс, выполняющий сортировку, получает свою часть файла от ведущего процесса и сортирует его. Ведущий процесс выполняет упорядочивание уже отсортированных блоков. При необходимости ведомым процессам может быть выделено более одного задания на сортировку. 

### Вспомогательные функции

In [ ]:
open System.Runtime.InteropServices
open Microsoft.FSharp.NativeInterop
open System.Text
open System.IO
open System

let rnd = Random()

type dw = uint
type lps = StringBuilder
type hdl = nativeint

let maxLength = 256

In [ ]:
let nums = Seq.initInfinite (fun _ -> string <| rnd.Next(-20, 50))
File.WriteAllText("files/unsorted.txt", Seq.initInfinite (fun _ -> nums |> Seq.take 5 |> String.concat " ")
                                                             |> Seq.take 60 |> String.concat "\n")

### Функции Win32 API
Ниже представлены использованные функции Win32 API для получения той или иной информации о системе и для задания некоторых параметров

In [ ]:
[<DllImport("kernel32.dll")>] extern dw GetCurrentThreadId()
[<DllImport("kernel32.dll")>] extern dw GetCurrentProcessId()
[<DllImport("kernel32.dll")>] extern void Sleep(dw ms)

let INVALID_HDL = nativeint 0xffffffff

type Protection = | RO = 0x02 | RW = 0x04
type MemoryAccess = | All = 0xf001fu
type FileAccess = | Read = 0x80000000u | Write = 0x40000000u | RW = 0xc0000000u

type ShareMode = | Zero = 0 | Read = 1 | Write = 2 | RW = 3
type CD = | CreateA = 2 | CreateN = 1 | OpenA = 4 | OpenE = 3

type Attribute = | Normal = 0x80

[<DllImport("kernel32.dll")>]
extern hdl CreateFile(string path, FileAccess acc, ShareMode sm, string, CD, Attribute, hdl)

[<DllImport("kernel32.dll")>]
extern bool WriteFile(hdl file, string buffer, int length, int, string)

[<DllImport("kernel32.dll")>]
extern hdl CreateFileMapping(hdl hFile, string sec, Protection px, dw maxSizeH, int maxSizeL, string name)
[<DllImport("kernel32.dll")>]
extern hdl OpenFileMapping(MemoryAccess access, bool inh, string name)

[<DllImport("kernel32.dll")>]
extern nativeint MapViewOfFile(hdl hFileM, MemoryAccess access, dw offsetH, dw offsetL, int numberOfBytes)

[<DllImport("kernel32.dll")>]
extern bool UnmapViewOfFile(hdl hFileM)

[<DllImport("kernel32.dll")>]
extern hdl CreateMailslot(string name, int size, int timeout, string)

[<DllImport("kernel32.dll")>]
extern hdl SetNamedPipeHandleState(hdl pipe, dw& mode, dw& maxCount, dw& timeout)

// [<DllImport("kernel32.dll")>]
// extern hdl CreateMailslot(string name, int size, int timeout, string)

[<DllImport("kernel32.dll")>] extern bool CloseHandle(hdl object)

[<DllImport("kernel32.dll")>] extern int GetLastError()

### Выполнение

In [ ]:
let file = @"files\unsorted.txt"
let name = @"Local\Lab4-num4"

open System.IO.MemoryMappedFiles

do
use view = MemoryMappedFile.CreateFromFile(@"files\abc.jx", FileMode.Open, "ImA")

let abc = @"\\.\mailslot\chunks" |> Seq.toArray
use ac = view.CreateViewAccessor(15, 140L)

ac.WriteArray(0L, abc, 0, abc.Length)


In [ ]:

let content = File.ReadAllLines(file)
let fMap = CreateFileMapping(0xffffffff, null, Protection.RW, 0u, maxLength, name)
let buffer = MapViewOfFile(fMap, MemoryAccess.All, 0u, 0u, maxLength)

let st = @"\\.\mailslot\chunks" |> Seq.toArray
Marshal.Copy(st, 0, buffer, st.Length)

let slotName = @"\\.\mailslot\chunks"

Sleep 4000u

let slot = CreateMailslot(slotName, 0, 0xffffffff, null)
let fHdl = CreateFile(slotName, FileAccess.Write, ShareMode.Read, null, CD.OpenE, Attribute.Normal, 0n)
let writeChunk (data: string) =
    if WriteFile(fHdl, data, (data.Length) * sizeof<char>, 0, null) |> not then
        failwith "Write error occured"
let count = 10

for t in content |> Seq.splitInto count do
    $"Sending: {t |> Array.toList}" |> display |> ignore
    writeChunk (t |> String.concat " ")
    Sleep 1000u

CloseHandle(fHdl); CloseHandle(slot) |> ignore

UnmapViewOfFile(buffer); 
CloseHandle(fMap) |> ignore

Sending: [11 31 46 40 -15; -13 -7 49 -18 2; 41 11 41 26 -10; ... ]

Sending: [-14 -20 -4 29 8; 29 12 -16 39 -17; 6 -10 5 6 7; ... ]

Sending: [-1 15 19 39 9; -2 4 21 23 11; -6 39 47 -16 38; ... ]

Sending: [32 25 43 9 -3; -7 43 47 36 8; 48 15 11 34 -11; ... ]

Sending: [35 39 41 32 48; 8 32 36 47 -8; 15 -17 45 18 24; ... ]

Sending: [18 -11 8 31 18; 8 -13 46 11 -7; 36 -16 11 -11 27; ... ]

Sending: [30 -11 47 0 32; 45 21 -9 -18 -12; 5 6 10 40 26; ... ]

Sending: [38 38 30 29 27; 5 8 -11 41 31; -13 22 38 17 22; ... ]

Sending: [14 11 -3 45 29; 17 0 17 7 46; 42 1 22 -13 19; ... ]

Sending: [15 8 24 11 -8; 25 -2 35 -8 30; -5 2 40 13 8; ... ]

In [ ]:
GetLastError()

##### Отображение файлов

In [ ]:
let name = @"Local\Lab4-num1"

let fMap = CreateFileMapping(INVALID_HDL, null, Protection.RW, 0u, maxLength, name)
let buffer = MapViewOfFile(fMap, MemoryAccess.All, 0u, 0u, maxLength)

let content = File.ReadAllLines(file)

let st = $"Длина файла: {content.Length}" |> Seq.toArray
Marshal.Copy(st, 0, buffer, st.Length)

Sleep 10000u

UnmapViewOfFile(buffer); CloseHandle(fMap) |> ignore

##### Почтовые ящики

##### Каналы

# Вывод

В результате лабораторной работы мной была проведена работа с механизмами межпроцессного взаимодействия, используя средства Win32 API.